In [36]:
// Some utility functions etc; stuff common to each day's puzzle

In [37]:
using System.IO;
using System.Text.RegularExpressions;
using SCG = System.Collections.Generic;

In [38]:
/// <summary>
/// Load a given day's puzzle input
/// </summary>
string[] LoadPuzzleInput(int year, int day)
{
    var filePath = $"Day{day}.txt";
    Console.WriteLine($"Loading puzzle file: {filePath}");
    
    var lines = File.ReadAllLines(filePath);

    Console.WriteLine($"Total lines: {lines.Length}");
    var maxLineLength = lines.Select(line => line.Length).Max();
    Console.WriteLine($"Max line length: {maxLineLength}");
    Console.WriteLine();

    return lines;
}

In [39]:
/// <summary>
/// Display a sample of the puzzle input lines
/// </summary>
void WriteLines<T>(IEnumerable<T> lines, int maxRows = 5, int maxCols = 500)
{
    foreach (var line in lines.Take(maxRows))
    {
        var lineStr = line.ToString();
        lineStr = (lineStr.Length - maxCols) switch {
            > 0 => lineStr.Substring(0, maxCols),
            _ => lineStr
        };
        Console.WriteLine(lineStr);
    }
}

In [40]:
/// <summary>
/// Exception thrown when <c>Ensure</c> check does not match
/// </summary>
class PuzzleFailureException(string message) : Exception(message) {}

/// <summary>
/// Ensure puzzle answer is correct
/// </summary>
void Ensure<T>(T expected, T answer) where T : IComparable<T>
{
    if (!expected.Equals(answer)) {
        throw new PuzzleFailureException($"Expected puzzle answer to equal {expected}, was {answer}");
    }
}

In [41]:
// Based on:
// https://github.com/dotnet/csharplang/discussions/388#discussioncomment-67632

/// <summmary>
/// Array deconstructor helper
/// </summary>
public static void Deconstruct<T>(this T[] array, out T item1)
{
    if (array is [var a]) {
        item1 = a;
    } else {
        throw new ArgumentException(nameof(array), $"Expected 1 item, got {array.Length}");
    }
}

/// <summmary>
/// Array deconstructor helper
/// </summary>
public static void Deconstruct<T>(this T[] array, out T item1, out T item2)
{
    if (array is [var a, var b])
    {
        item1 = a;
        item2 = b;
    } else {
        throw new ArgumentException(nameof(array), $"Expected 2 items, got {array.Length}");
    }
}

In [42]:
using System.Text.RegularExpressions;

/// <summary>
/// Regex deconstructor helper - assign values corresponding to gc[i].Value, or null if no group
/// </summary>
public static void Deconstruct(this GroupCollection gc, out string a, out string b, out string c, out string d)
{
    var values = Enumerable.Range(0, 4).Select(i => i < gc.Count ? gc[i].Value : null).ToArray();
    a = values[0];
    b = values[1];
    c = values[2];
    d = values[3];
}

/// <summary>
/// Regex deconstructor helper - assign values corresponding to gc[i].Value, or null if no group
/// </summary>
public static void Deconstruct(this GroupCollection gc, out string a, out string b, out string c)
{
    (a, b, c, _) = gc;
}

In [43]:
/// <summary>
/// Separate <c>source</c> into multiple groups
/// </summary>
/// <remarks>Useful when the puzzle input contains multiple sections</remarks>
public static IEnumerable<string[]> SeparateBy(this IEnumerable<string> source, Func<string, bool> separator)
{
    ArgumentNullException.ThrowIfNull(source);
    ArgumentNullException.ThrowIfNull(separator);

    List<string> current = new();
    foreach (var line in source)
    {
        if (separator(line))
        {
            yield return current.ToArray();
            current = new();
            continue;
        }
        
        current.Add(line);
    }
    yield return current.ToArray();
}

In [44]:
// Waiting for dotnet-interactive to update to .NET 9...
// https://github.com/dotnet/interactive/issues/3760

if (Environment.Version.Major >= 9) {
    throw new Exception("We don't need the following extensions anymore!");
}

/// <summary>
/// Enumerate a collection alongside its index
/// </summary>
/// <remarks>Analogue of System.Linq.Enumerable.Index in .NET 9</remarks>
public static IEnumerable<(int Index, T Item)> Index<T>(this IEnumerable<T> source)
{
    var i = 0;
    foreach (var item in source)
    {
        yield return (i++, item);
    }
}

In [45]:
// Points and Grids
/////

record struct Point(int X, int Y)
{
    public override string ToString() => $"({X}, {Y})";
    
    public static Point operator +(Point a, Point b) => (a.X + b.X, a.Y + b.Y);
    public static Point operator -(Point a, Point b) => a + (-b);
    public static Point operator -(Point p) => (-p.X, -p.Y);
    public static Point operator *(Point p, int x) => (p.X * x, p.Y * x);

    public static implicit operator Point((int x, int y) xy) => new(xy.x, xy.y);
}

static Point Up = (0, -1);
static Point Down = -Up;
static Point Left = (-1, 0);
static Point Right = -Left;

Dictionary<Point, Point> RightTurns = new()
{
    { Up, Right },
    { Right, Down },
    { Down, Left },
    { Left, Up }
};
Point TurnRight(Point p) => RightTurns[p];

In [46]:
// Tests
if (false) 
{
    Point p = new(1, 1);
    Console.WriteLine(p);

    foreach (var testPoint in new[] {Up, Down, Left, Right})
    {
        Console.WriteLine(p + testPoint);
    }

    Console.WriteLine(p * 5);
}

In [47]:
class CharGrid(string[] inputLines)
{
    string[] _inputLines = inputLines;

    /// <summary>
    /// Maximum addressible row
    /// </summary>
    public int Rows { get; } = inputLines.Length;

    /// <summary>
    /// Maximum addressible column
    /// </summary>
    public int Cols { get; } = inputLines[0].Length;

    /// <summary>
    /// Character at corresponding point
    /// </summary>
    public char this[Point p]
    {
        get => _inputLines[p.Y][p.X];
        set
        {
            var lineBuffer = _inputLines[p.Y].ToCharArray();
            lineBuffer[p.X] = value;
            _inputLines[p.Y] = new string(lineBuffer);
        }
    }

    /// <summary>
    /// Enumerate all points and characters in the grid
    /// </summary>
    /// <remarks>Useful for searching for specific features</remarks>
    public IEnumerable<(Point point, char ch)> Enumerate()
    {
        var q = from y in Enumerable.Range(0, Rows)
                from x in Enumerable.Range(0, Cols)
                let p = new Point(x, y)
                select (p, this[p]);
        return q;
    }

    /// <summary>
    /// Is <c>p</c> a valid point within the grid?
    /// </summary>
    public bool IsValid(Point p) => p switch
    {
        (< 0, _) => false,
        (_, < 0) => false,
        (var x, _) when x >= Cols => false,
        (_, var y) when y >= Rows => false,
        _ => true
    };

    public CharGrid Clone() => new([.._inputLines]);
}

In [48]:
static Dictionary<char, int> DigitLookup = new()
{
    { '0', 0 },
    { '1', 1 },
    { '2', 2 },
    { '3', 3 },
    { '4', 4 },
    { '5', 5 },
    { '6', 6 },
    { '7', 7 },
    { '8', 8 },
    { '9', 9 }
};

/// <summary>
/// Convert a number character into the numeric value. A common puzzle requirement.
/// </summary>
public static int ToInt(this char ch) => DigitLookup[ch];

In [49]:
/// <summary>
/// Helper to ensure loops always terminate
/// </summary>
class SafetyLimit
{
    int _attempts = 0;

    public void EnsureBelow(int limit)
    {
        if (++_attempts == limit)
        {
            throw new Exception($"Safety limit of {limit} exceeded.");
        }
    }
}

In [50]:
// Tests

if (false)
{
    string[] testInputLines = [
        "ab",
        "cd"
    ];

    CharGrid grid = new(testInputLines);

    Point p = new(0, 0);
    Console.WriteLine(grid[p]);
    Console.WriteLine(grid[p + Right]);
    Console.WriteLine(grid[p + Down]);
    Console.WriteLine(grid[p + Down + Right]);

    Console.WriteLine(grid.IsValid(p));
    Console.WriteLine(grid.IsValid(p + Right));
    Console.WriteLine(grid.IsValid(p + Right + Right));
}

In [1]:
static IEnumerable<LinkedListNode<T>> WalkNodes<T>(this LinkedList<T> list)
{
    var node = list.First;
    while (node != null)
    {
        yield return node;
        node = node.Next;
    }
}

In [ ]:
/// <summary>
/// Breadth-First Search, another common puzzle algorithm.
/// </summary>
IEnumerable<T> BFS<T>(T startNode, Func<T, IEnumerable<T>> getNeighbours)
{
    HashSet<T> visited = new();
    
    Queue<T> queue = new();
    queue.Enqueue(startNode);

    while (queue.TryDequeue(out var node))
    {
        if (visited.Contains(node)) continue;

        yield return node;
        
        visited.Add(node);

        foreach (var neighbour in getNeighbours(node))
        {
            queue.Enqueue(neighbour);
        }
    }
}